In [9]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata

import vertexai
import vertexai.generative_models as genai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig
import json
import time
import os
import pandas as pd
import numpy as np 
from dotenv import load_dotenv
load_dotenv(verbose=True)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../input/mtec-unstructured-data-390708-c264fdc6ca7a.json"
vertexai.init()
# モデルの準備
gemini = genai.GenerativeModel("models/gemini-1.5-flash")

from pydantic import BaseModel, Field
from openai import AzureOpenAI
engine='gpt-4o-mini'
gpt = AzureOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.environ.get("OPENAI_ENDPOINT"),
)

In [10]:
def load_data(path):
    with open(path, 'rb') as f:
        data = pd.DataFrame().from_dict(json.load(f)).T.reset_index(names="question_number")
        f.close()
    return data

In [11]:
df_train = load_data("../train/questions.json")
df_train['flag'] = 'train'
df_eval = load_data("../valid/questions.json")
df_eval['flag'] = 'valid'
df_test = load_data("../test/questions.json")
df_test['flag'] = 'test'
df = pd.concat([df_train, df_eval, df_test], axis=0)

In [12]:
def create_toc(flag, doc_id):
    path = f"../{flag}/reports/{doc_id}"
    file_lst = os.listdir(path)
    toc = {}
    for file in file_lst:
        with open(f"{path}/{file}", "r") as f:
            html = f.read()
            f.close()
        soup = bs(html, 'html.parser')
        tables = soup.find_all("table") 
        for table in tables:
            header = table.find_parent().find_previous_sibling(['h1', 'h2', 'h3', 'h4','h5','h6'])
            if header:
                text = unicodedata.normalize('NFKC', header.text).replace('\n', '').replace(' ', '')
                toc[table.get('table-id')] = text
            else:
                try:
                    toc[table.get('table-id')] = text
                except:
                    toc[table.get('table-id')] = None
    return toc

In [ ]:

toc_dct = {}
for index, row in df.drop_duplicates(subset='doc_id', ).iterrows():
    toc = create_toc(row['flag'], row['doc_id'])
    toc_dct[row['doc_id']] = toc

In [15]:
with open('toc_dct.json', 'w') as f:
    json.dump(toc_dct, f, indent=4) 

In [95]:
class TQAtast():
    def __init__(self, question, doc_id, flag, train_dct=train_dct):
        self.question = unicodedata.normalize('NFKC', question)
        self.doc_id = doc_id
        self.flag = flag
        self.file_path = f'../{self.flag}/reports/{self.doc_id}/'
        self.train_dct   = train_dct 
    
    def get_words(self,):
        if re.search(r"「(.*)」", self.question):
            words = re.search(r"「(.*)」", self.question).group(1)
            words = re.sub(r"\(.*\)", '', words)
            print(words)
            words = re.split(r"、|-|又は|及び", words)
            words = [word for word in words if word != '']

    
        if '個別' in self.question:
            words.append('個別')
        elif '連結' in self.question:
            words.append('連結')
        return words
    
    def related_tables(self,):
        words = self.get_words()
        with open('toc_dct.json', 'r') as f:
            toc_dct = json.load(f)
        max_related_item = max(self.train_dct, key=lambda item: sum(word in item['question'] for word in words)) 
        my_toc = self.create_toc(self.flag, self.doc_id)
        return toc, my_toc 


In [96]:
row = df_eval.iloc[1]
task = TQAtast(row['question'], row['table_id'], row['flag'])
related_tables = task.related_tables()
print(row['question'])
print(related_tables)

当期純利益又は当期純損失、経営指標等


FileNotFoundError: [Errno 2] No such file or directory: '../eval/reports/S100TUGR-0101010-tab3'